# 03 - Structured Outputs and JSON

## Objectives
- JSON schema design for interview questions
- Structured output formats (questions, answers, feedback)
- Response parsing and validation
- Error handling for malformed outputs

## Expected Output
JSON schemas and parsing utilities

In [1]:
import sys
sys.path.append('../src')

from core.llm_client import setup_openai_client
from core.schemas import create_question_schema, create_response_schema
from core.llm_integration import implement_structured_generation
from utils.json_utils import implement_schema_validation, parse_llm_responses

[JSON] Creating question schema
Question Schema Structure:
┌─ Required Fields:
│  ✓ question (string)
│  ✓ type (string)
│  ✓ difficulty (string)
│  ✓ duration_minutes (integer)
├─ Optional Fields:
│  • follow_ups (array)
│  • evaluation_points (array)
├─ Interview Types:
│  1. technical
│  2. behavioral
│  3. system_design
│  4. coding
└─ Difficulty Levels:
   1. junior
   2. mid
   3. senior
   4. lead
[SCHEMA] Question schema v1.0 created successfully
[JSON] Creating response schema
Response Schema Structure:
┌─ Required Fields:
│  ✓ response_text (string)
│  ✓ overall_rating (string)
│  ✓ feedback (object)
├─ Feedback Structure:
│  • summary (string)
│  • strengths (array)
│  • improvements (array)
├─ Rating Scales:
│  1. excellent
│  2. good
│  3. fair
│  4. poor
└─ Evaluation Categories:
   1. technical_accuracy
   2. communication
   3. problem_solving
   4. depth
[SCHEMA] Response schema v1.0 created successfully


## Phase 1: Core Schema Design

In [2]:
question_schema_config = create_question_schema()

[JSON] Creating question schema
Question Schema Structure:
┌─ Required Fields:
│  ✓ question (string)
│  ✓ type (string)
│  ✓ difficulty (string)
│  ✓ duration_minutes (integer)
├─ Optional Fields:
│  • follow_ups (array)
│  • evaluation_points (array)
├─ Interview Types:
│  1. technical
│  2. behavioral
│  3. system_design
│  4. coding
└─ Difficulty Levels:
   1. junior
   2. mid
   3. senior
   4. lead
[SCHEMA] Question schema v1.0 created successfully


In [3]:
response_schema_config = create_response_schema()

[JSON] Creating response schema
Response Schema Structure:
┌─ Required Fields:
│  ✓ response_text (string)
│  ✓ overall_rating (string)
│  ✓ feedback (object)
├─ Feedback Structure:
│  • summary (string)
│  • strengths (array)
│  • improvements (array)
├─ Rating Scales:
│  1. excellent
│  2. good
│  3. fair
│  4. poor
└─ Evaluation Categories:
   1. technical_accuracy
   2. communication
   3. problem_solving
   4. depth
[SCHEMA] Response schema v1.0 created successfully


## Phase 2: Generation & Validation

In [4]:
client = setup_openai_client("../.env")

[SETUP] Initializing OpenAI client configuration
[CONFIG] Loading environment variables from: ../.env
[CONFIG] API key successfully retrieved from environment
[SETUP] OpenAI client initialized successfully


In [5]:
test_prompt = "Generate a technical interview question about Python data structures for a mid-level developer."
generation_result = implement_structured_generation(
    client, 
    test_prompt, 
    question_schema_config["schema"]
)

if generation_result["success"]:
    print(f"[TEST] Generated question: {generation_result['data']['question'][:100]}...")
    
print("[COMPLETED] Phase 2.1 - Structured Generation")

[JSON] Starting structured generation with gpt-4o-mini
[SUCCESS] Generated 6 fields, 293 tokens
[TEST] Generated question: Explain the differences between lists, tuples, and sets in Python. In what scenarios would you choos...
[COMPLETED] Phase 2.1 - Structured Generation


In [6]:
question_validator = implement_schema_validation(question_schema_config)

test_question = {
    "question": "What is the difference between == and === in JavaScript?",
    "type": "technical",
    "difficulty": "mid",
    "duration_minutes": 5,
    "follow_ups": ["Can you explain type coercion?"],
    "evaluation_points": ["Technical accuracy", "Clear explanation"]
}

validation_result = question_validator["validate_single"](test_question)
print(f"[TEST] Validation result: {validation_result['is_valid']}")

print("[COMPLETED] Phase 2.2 - Schema Validation")

[JSON] Creating schema validator
[COMPLETED] Schema validator created for 4 required fields
[TEST] Validation result: True
[COMPLETED] Phase 2.2 - Schema Validation


In [7]:
test_malformed = '{"question": "What is Python?", "type": "technical", "difficulty": "junior",}'  # trailing comma
parse_result = parse_llm_responses(test_malformed, question_schema_config["schema"])

print(f"Parse result: {'SUCCESS' if parse_result['success'] else 'FAILED'}")
print("[COMPLETED] Phase 3.1 - LLM Response Parsing")

[PARSE] Processing LLM response of 77 characters
[ERROR] JSON parsing failed: Expecting property name enclosed in double quotes: line 1 column 77 (char 76)
[REPAIR] Attempting JSON repair
[REPAIR] Attempting to fix malformed JSON
[REPAIR] Applied fixes: trailing_commas, quote_keys
[SUCCESS] JSON repaired successfully
[WARNING] Schema validation failed: 'duration_minutes' is a required property

Failed validating 'required' in schema:
    {'type': 'obj...
Parse result: SUCCESS
[COMPLETED] Phase 3.1 - LLM Response Parsing
